In [1]:
%pip install pdf2image


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: C:\Python310\python.exe -m pip install --upgrade pip


In [8]:
%pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
  Using cached pdfplumber-0.10.2-py3-none-any.whl (47 kB)
  Using cached pypdfium2-4.18.0-py3-none-win_amd64.whl (2.7 MB)
  Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Using cached cryptography-41.0.3-cp37-abi3-win_amd64.whl (2.6 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Laxmi\\AppData\\Roaming\\Python\\Python39\\site-packages\\~il\\_imaging.cp39-win_amd64.pyd'
Check the permissions.

You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
%pip install -U spacy -q

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
easyocr 1.6.2 requires PyYAML, which is not installed.
easyocr 1.6.2 requires scikit-image, which is not installed.
You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
%python -m spacy info

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [6]:
!apt-get install -y poppler-utils
!apt-get install -y python-poppler-qt5

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
import os
from pdf2image import convert_from_path
import pytesseract
import json


In [12]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [13]:
# Function to convert PDF to images and save them in a folder
def pdf_to_images(pdf_file_path, output_folder):
    images = convert_from_path(pdf_file_path)

    for i, image in enumerate(images):
        image_path = os.path.join(output_folder, f"page_{i + 1}.jpg")
        image.save(image_path, "JPEG")

    return images

In [14]:
def extract_text_from_image(path_to_img):

  try:
    #now we'll get the text from image through tesserac
    return pytesseract.image_to_string((path_to_img))
  except:
    return ' '

In [17]:
f = open('annotations (1).json')
TRAIN_DATA = json.load(f)

In [18]:
TRAIN_DATA

{'classes': ['NAME', 'ADDRESS', 'GENDER', 'NUMBER', 'DOB'],
 'annotations': [['Laxmi Laxmikant Kurapati  Female  28-04-2003   1512,Daji peth Solapur   1233 3242 3233 \r',
   {'entities': [[0, 24, 'NAME'],
     [26, 32, 'GENDER'],
     [34, 44, 'DOB'],
     [47, 69, 'ADDRESS'],
     [72, 86, 'NUMBER']]}],
  ['\r', {'entities': []}],
  ['Prem Bhimsa Jhadav Male 04-08-2002 1344 Nevi peth Solapur 3454 5433 2324\r',
   {'entities': [[0, 18, 'NAME'],
     [19, 23, 'GENDER'],
     [24, 34, 'DOB'],
     [40, 57, 'ADDRESS'],
     [58, 72, 'NUMBER']]}],
  ['\r', {'entities': []}],
  ['Amita Laxmikant Kurapati Female 22-12-1995  1512, Daji peth Solapur 34323 3424 3554 \r',
   {'entities': [[0, 24, 'NAME'],
     [25, 31, 'GENDER'],
     [32, 42, 'DOB'],
     [44, 67, 'ADDRESS'],
     [68, 83, 'NUMBER']]}],
  ['\r', {'entities': []}],
  ['Chaitan Laxmikant Kurapati Male 22-12-1995  1512, Daji peth Solapur 34323 3424 3554 \r',
   {'entities': [[0, 26, 'NAME'],
     [27, 31, 'GENDER'],
     [32, 42, 

In [19]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object


100%|██████████| 52/52 [00:00<00:00, 1270.73it/s]


In [20]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .

[2023-08-02 15:53:58,246] [INFO] Set up nlp object from config
[2023-08-02 15:53:58,274] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-08-02 15:53:58,283] [INFO] Created vocabulary
[2023-08-02 15:53:58,288] [INFO] Finished initializing nlp object
[2023-08-02 15:53:58,747] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     53.68    0.00    0.00    0.00    0.00
 50     200         89.60   2437.87  100.00  100.00  100.00    1.00
116     400          0.62      0.74  100.00  100.00  100.00    1.00
189     600          0.00      0.00  100.00  100.00  100.00    1.00
289     800          0.00      0.00  100.00  100.00  100.00    1.00
389    1000          0.00      0.00  100.00  100.00  100.00    1.00
523    1200          0.00      0.00  100.00  100.00  100.00    1.00
723    1400          0.00      0.00  100.00  100.00  100.00    1.00
923    1600          0.00      0.00  100.00  100.00  100.00    1.00

In [24]:
nlp_ner = spacy.load("model-best")

In [25]:

def extract_named_entities(text):
    # doc = nlp(text)
    # entities = [(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in doc.ents]
    # return entities
    ner_results = nlp_ner(text)
    entities = [(ent.text, ent.label_, ent.start_char, ent.end_char) for ent in ner_results.ents]
    print(entities)
    return ner_results

In [28]:
pdf_file_path = "1.pdf"  # Replace with the path of your PDF file
output_folder = "output"

if not os.path.exists(output_folder):
        os.makedirs(output_folder)

images = pdf_to_images(pdf_file_path, output_folder)

for i, image in enumerate(images):
        print(f"Processing Page {i + 1}")
        # Perform OCR on the image to extract text
        text = extract_text_from_image(image)

        # Perform NER on the extracted text to identify named entities
        entities = extract_named_entities(text)

        spacy.displacy.render(entities, style="ent", jupyter=True) # display in Jupyter

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001E55A405D10>